# AV-PINO Model Training

This notebook demonstrates the complete training pipeline for the AV-PINO motor fault diagnosis system, including:

- Physics-informed loss functions
- Multi-physics coupling
- Uncertainty quantification training
- Training monitoring and visualization

In [ ]:
# Import required libraries
import sys
import os
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pathlib import Path
import time

# Import AV-PINO components
from config.experiment_config import ExperimentManager
from data.cwru_loader import CWRUDataLoader
from data.preprocessor import DataPreprocessor
from training.training_engine import TrainingEngine
from training.physics_informed_loss import PhysicsInformedLoss
from training.advanced_loss_functions import AdvancedLossManager
from validation.benchmarking_suite import BenchmarkingSuite
from visualization.visualization_manager import VisualizationManager

print("Training components imported successfully!")

## 1. Experiment Configuration

In [ ]:
# Setup experiment configuration
config_manager = ExperimentManager()
config = config_manager.create_default_config()

# Training-specific configuration
config.training.update({
    "epochs": 100,
    "batch_size": 32,
    "learning_rate": 1e-3,
    "optimizer": "adamw",
    "scheduler": "cosine",
    "weight_decay": 1e-4,
    "gradient_clip": 1.0
})

# Physics loss configuration
config.physics["loss_weights"] = {
    "data": 1.0,
    "physics": 0.1,
    "consistency": 0.05,
    "variational": 0.01
}

# Setup reproducibility
final_config = config_manager.setup_experiment()

print(f"Training configuration:")
print(f"  Epochs: {final_config.training['epochs']}")
print(f"  Batch size: {final_config.training['batch_size']}")
print(f"  Learning rate: {final_config.training['learning_rate']}")
print(f"  Physics loss weights: {final_config.physics['loss_weights']}")

## 2. Data Preparation

In [ ]:
# Initialize data components
data_loader = CWRUDataLoader()
preprocessor = DataPreprocessor(final_config)

# Load and preprocess data
try:
    # Load CWRU dataset
    train_data, val_data, test_data = data_loader.load_dataset(
        data_path="../data/cwru",
        download=True,
        train_split=0.7,
        val_split=0.15,
        test_split=0.15
    )
    
    print(f"Dataset loaded successfully:")
    print(f"  Training samples: {len(train_data)}")
    print(f"  Validation samples: {len(val_data)}")
    print(f"  Test samples: {len(test_data)}")
    
except Exception as e:
    print(f"Using synthetic data for demonstration: {e}")
    
    # Create synthetic dataset
    train_data = data_loader.create_synthetic_dataset(1000)
    val_data = data_loader.create_synthetic_dataset(200)
    test_data = data_loader.create_synthetic_dataset(200)

# Preprocess data
train_processed = preprocessor.preprocess_dataset(train_data)
val_processed = preprocessor.preprocess_dataset(val_data)

# Create data loaders
train_loader = DataLoader(
    train_processed, 
    batch_size=final_config.training["batch_size"],
    shuffle=True,
    num_workers=4
)

val_loader = DataLoader(
    val_processed,
    batch_size=final_config.training["batch_size"],
    shuffle=False,
    num_workers=4
)

print(f"Data loaders created:")
print(f"  Training batches: {len(train_loader)}")
print(f"  Validation batches: {len(val_loader)}")

## 3. Model and Loss Function Setup

In [ ]:
# Initialize training engine
training_engine = TrainingEngine(final_config)

# Get model architecture
model = training_engine.create_model()
print(f"Model created: {model.__class__.__name__}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

# Initialize physics-informed loss
physics_loss = PhysicsInformedLoss(final_config)
advanced_loss = AdvancedLossManager(final_config)

print(f"Loss functions initialized:")
print(f"  Physics constraints: {physics_loss.constraint_types}")
print(f"  Loss weights: {physics_loss.loss_weights}")

# Setup optimizer and scheduler
optimizer = training_engine.create_optimizer(model)
scheduler = training_engine.create_scheduler(optimizer)

print(f"Optimizer: {optimizer.__class__.__name__}")
print(f"Scheduler: {scheduler.__class__.__name__}")

## 4. Training Loop with Physics Constraints

In [ ]:
# Training monitoring
training_history = {
    'train_loss': [],
    'val_loss': [],
    'data_loss': [],
    'physics_loss': [],
    'consistency_loss': [],
    'accuracy': [],
    'physics_violations': []
}

# Training loop
print("Starting training...")
start_time = time.time()

for epoch in range(final_config.training["epochs"]):
    # Training phase
    model.train()
    train_metrics = training_engine.train_epoch(
        model, train_loader, optimizer, physics_loss, epoch
    )
    
    # Validation phase
    model.eval()
    val_metrics = training_engine.validate_epoch(
        model, val_loader, physics_loss, epoch
    )
    
    # Update learning rate
    scheduler.step()
    
    # Record metrics
    training_history['train_loss'].append(train_metrics['total_loss'])
    training_history['val_loss'].append(val_metrics['total_loss'])
    training_history['data_loss'].append(train_metrics['data_loss'])
    training_history['physics_loss'].append(train_metrics['physics_loss'])
    training_history['consistency_loss'].append(train_metrics['consistency_loss'])
    training_history['accuracy'].append(val_metrics['accuracy'])
    training_history['physics_violations'].append(train_metrics['physics_violations'])
    
    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{final_config.training['epochs']}:")
        print(f"  Train Loss: {train_metrics['total_loss']:.4f}")
        print(f"  Val Loss: {val_metrics['total_loss']:.4f}")
        print(f"  Val Accuracy: {val_metrics['accuracy']:.4f}")
        print(f"  Physics Violations: {train_metrics['physics_violations']:.6f}")
        print(f"  Learning Rate: {scheduler.get_last_lr()[0]:.6f}")

training_time = time.time() - start_time
print(f"\nTraining completed in {training_time:.2f} seconds")
print(f"Final validation accuracy: {training_history['accuracy'][-1]:.4f}")

## 5. Training Visualization

In [ ]:
# Plot training curves
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Loss curves
axes[0, 0].plot(training_history['train_loss'], label='Train Loss')
axes[0, 0].plot(training_history['val_loss'], label='Val Loss')
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True)

# Component losses
axes[0, 1].plot(training_history['data_loss'], label='Data Loss')
axes[0, 1].plot(training_history['physics_loss'], label='Physics Loss')
axes[0, 1].plot(training_history['consistency_loss'], label='Consistency Loss')
axes[0, 1].set_title('Loss Components')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)
axes[0, 1].set_yscale('log')

# Accuracy
axes[0, 2].plot(training_history['accuracy'])
axes[0, 2].set_title('Validation Accuracy')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('Accuracy')
axes[0, 2].grid(True)

# Physics violations
axes[1, 0].plot(training_history['physics_violations'])
axes[1, 0].set_title('Physics Constraint Violations')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Violation Magnitude')
axes[1, 0].grid(True)
axes[1, 0].set_yscale('log')

# Learning rate schedule
lr_history = [scheduler.get_last_lr()[0] for _ in range(len(training_history['train_loss']))]
axes[1, 1].plot(lr_history)
axes[1, 1].set_title('Learning Rate Schedule')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Learning Rate')
axes[1, 1].grid(True)
axes[1, 1].set_yscale('log')

# Loss ratio analysis
physics_ratio = np.array(training_history['physics_loss']) / np.array(training_history['data_loss'])
axes[1, 2].plot(physics_ratio)
axes[1, 2].set_title('Physics/Data Loss Ratio')
axes[1, 2].set_xlabel('Epoch')
axes[1, 2].set_ylabel('Ratio')
axes[1, 2].grid(True)

plt.tight_layout()
plt.show()

print("Training visualization complete!")

## 6. Model Evaluation and Physics Validation

In [ ]:
# Evaluate trained model
benchmarking_suite = BenchmarkingSuite(final_config)

# Test data evaluation
test_processed = preprocessor.preprocess_dataset(test_data)
test_loader = DataLoader(
    test_processed,
    batch_size=final_config.training["batch_size"],
    shuffle=False
)

# Comprehensive evaluation
evaluation_results = benchmarking_suite.evaluate_model(
    model, test_loader, physics_loss
)

print("Model Evaluation Results:")
print(f"  Test Accuracy: {evaluation_results['accuracy']:.4f}")
print(f"  Test Loss: {evaluation_results['total_loss']:.4f}")
print(f"  Physics Consistency: {evaluation_results['physics_consistency']:.4f}")
print(f"  Inference Latency: {evaluation_results['inference_latency']:.2f} ms")

# Physics validation
physics_validation = benchmarking_suite.validate_physics_constraints(
    model, test_loader
)

print("\nPhysics Validation:")
for constraint, violation in physics_validation.items():
    print(f"  {constraint}: {violation:.6f}")

# Fault classification performance
classification_results = benchmarking_suite.evaluate_fault_classification(
    model, test_loader
)

print("\nFault Classification Results:")
for fault_type, metrics in classification_results.items():
    print(f"  {fault_type}:")
    print(f"    Precision: {metrics['precision']:.4f}")
    print(f"    Recall: {metrics['recall']:.4f}")
    print(f"    F1-Score: {metrics['f1']:.4f}")

## 7. Model Saving and Export

In [ ]:
# Save trained model
model_save_path = "../models/av_pino_trained.pth"
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# Save model state
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'config': final_config,
    'training_history': training_history,
    'evaluation_results': evaluation_results,
    'epoch': final_config.training['epochs']
}, model_save_path)

print(f"Model saved to: {model_save_path}")

# Save training configuration
config_save_path = "../models/training_config.yaml"
config_manager.save_config(final_config, config_save_path)
print(f"Configuration saved to: {config_save_path}")

# Export model for deployment
try:
    # Export to ONNX for edge deployment
    dummy_input = torch.randn(1, 1, 1024)
    onnx_path = "../models/av_pino_model.onnx"
    
    torch.onnx.export(
        model,
        dummy_input,
        onnx_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    )
    print(f"ONNX model exported to: {onnx_path}")
    
except Exception as e:
    print(f"ONNX export failed: {e}")

print("\nModel training and export complete!")

## 8. Training Summary and Analysis

In [ ]:
# Generate training summary
print("=" * 60)
print("AV-PINO TRAINING SUMMARY")
print("=" * 60)

print(f"\nModel Architecture:")
print(f"  Type: {final_config.model['architecture']}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"  Modes: {final_config.model['modes']}")
print(f"  Width: {final_config.model['width']}")

print(f"\nTraining Configuration:")
print(f"  Epochs: {final_config.training['epochs']}")
print(f"  Batch Size: {final_config.training['batch_size']}")
print(f"  Learning Rate: {final_config.training['learning_rate']}")
print(f"  Optimizer: {final_config.training['optimizer']}")
print(f"  Training Time: {training_time:.2f} seconds")

print(f"\nPhysics Integration:")
print(f"  Constraints: {final_config.physics['constraints']}")
print(f"  Loss Weights: {final_config.physics['loss_weights']}")
print(f"  Final Physics Violations: {training_history['physics_violations'][-1]:.6f}")

print(f"\nPerformance Results:")
print(f"  Final Training Loss: {training_history['train_loss'][-1]:.4f}")
print(f"  Final Validation Loss: {training_history['val_loss'][-1]:.4f}")
print(f"  Final Validation Accuracy: {training_history['accuracy'][-1]:.4f}")
print(f"  Test Accuracy: {evaluation_results['accuracy']:.4f}")
print(f"  Physics Consistency: {evaluation_results['physics_consistency']:.4f}")
print(f"  Inference Latency: {evaluation_results['inference_latency']:.2f} ms")

print(f"\nModel Files:")
print(f"  Model: {model_save_path}")
print(f"  Configuration: {config_save_path}")
if 'onnx_path' in locals():
    print(f"  ONNX Export: {onnx_path}")

print("\n" + "=" * 60)
print("Training completed successfully!")
print("Ready for real-time inference and deployment.")
print("=" * 60)